# 1. Init 

In [1]:
from qdrant_client import QdrantClient, models
from qdrant_client import models
from datetime import datetime

# Decide which dense encoding model to use 
model_handle = "jinaai/jina-embeddings-v2-small-en"

/workspaces/reddit_search/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient("http://localhost:6333")

In [3]:
import pandas as pd 
import numpy as np 

In [4]:
reddit_df = pd.read_csv("/workspaces/reddit_search/data/reddit_posts_and_comments.csv")
reddit_df.head() 

,post_title,post_text,subreddit,post_author,post_url,post_upvotes,post_downvotes,comment_upvotes,comment_downvotes,comment_text,comment_author
0,[D] - NeurIPS'2025 Reviews,"Hey everyone,\n\nNeurIPS 2025 reviews should b...",MachineLearning,Proof-Marsupial-5367,https://www.reddit.com/r/MachineLearning/comme...,203,0,77,0,Friendly reminder that reviews this year are s...,ChoiceStranger2898
1,[D] - NeurIPS'2025 Reviews,"Hey everyone,\n\nNeurIPS 2025 reviews should b...",MachineLearning,Proof-Marsupial-5367,https://www.reddit.com/r/MachineLearning/comme...,203,0,36,0,I had a dream recently where my upcoming avera...,popeldo
2,[D] - NeurIPS'2025 Reviews,"Hey everyone,\n\nNeurIPS 2025 reviews should b...",MachineLearning,Proof-Marsupial-5367,https://www.reddit.com/r/MachineLearning/comme...,203,0,63,0,I will treat the scores as a divine interventi...,matcha-coconut
3,[D] - NeurIPS'2025 Reviews,"Hey everyone,\n\nNeurIPS 2025 reviews should b...",MachineLearning,Proof-Marsupial-5367,https://www.reddit.com/r/MachineLearning/comme...,203,0,33,0,"July 24, so as long as it's July 24 somewhere ...",SmolLM
4,[D] - NeurIPS'2025 Reviews,"Hey everyone,\n\nNeurIPS 2025 reviews should b...",MachineLearning,Proof-Marsupial-5367,https://www.reddit.com/r/MachineLearning/comme...,203,0,34,0,"Well, if you feel heart-broken, be assured tha...",Marionberry6886


In [5]:
# combine 
reddit_df['post_title_text'] = reddit_df['post_title'] + '-' + reddit_df['post_text'] 

In [6]:
client.delete_collection("reddit_post")
client.delete_collection("reddit_comment")

True

In [7]:
# set up collection 
from qdrant_client import models

# Define the collection name
collection_name = "reddit_post"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=512,  # for sentence-transformers embeddings
        distance=models.Distance.COSINE
    ),
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

client.get_collections()


CollectionsResponse(collections=[CollectionDescription(name='reddit_post')])

In [8]:
# Create the collection with specified vector parameters

collection_name = "reddit_comment"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=512,  # for sentence-transformers embeddings
        distance=models.Distance.COSINE
    ),
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

client.get_collections()


CollectionsResponse(collections=[CollectionDescription(name='reddit_post'), CollectionDescription(name='reddit_comment')])

In [12]:
# Truncate long text 
def truncate_text(text, max_length=4000):
    """Truncate text but try to end at sentence boundary."""
    if len(text) <= max_length:
        return text
    
    # Truncate and try to end at sentence
    truncated = text[:max_length]
    last_period = truncated.rfind('.')
    last_space = truncated.rfind(' ')
    
    # End at sentence if period found in last 200 chars
    if last_period > max_length - 200:
        return truncated[:last_period + 1]
    # Otherwise end at word boundary
    elif last_space > max_length - 50:
        return truncated[:last_space]
    else:
        return truncated

In [13]:
from qdrant_client import models
from datetime import datetime

# Decide which dense encoding model to use 
model_handle = "jinaai/jina-embeddings-v2-small-en"


filtered_points = []
skipped_empty = 0
truncated_count = 0
id = 0

for idx, row in reddit_df.iterrows():
    # Combine title and text
    title = str(row['post_title']) if pd.notna(row['post_title']) else ""
    text = str(row['post_text']) if pd.notna(row['post_text']) else ""
    combined_text = f"{title}. {text}".strip(". ")
    
    # Skip if essentially empty
    if not combined_text or combined_text == "No content":
        skipped_empty += 1
        continue
    
    # Truncate if too long
    original_length = len(combined_text)
    combined_text = truncate_text(combined_text, max_length=4000)
    
    if len(combined_text) < original_length:
        truncated_count += 1
    
    point = models.PointStruct(
        id=id,
        vector=models.Document(
            text=combined_text, 
            model=model_handle
        ),
        payload={
            "text": combined_text,
            "post_title": title,
            "post_text": str(row['post_text']) if pd.notna(row['post_text']) else "",
            "subreddit": str(row['subreddit']) if pd.notna(row['subreddit']) else "",
            "post_author": str(row['post_author']) if pd.notna(row['post_author']) else "",
            "post_url": str(row['post_url']) if pd.notna(row['post_url']) else "",
            "post_upvotes": int(row['post_upvotes']) if pd.notna(row['post_upvotes']) else 0,
            "post_downvotes": int(row['post_downvotes']) if pd.notna(row['post_downvotes']) else 0,
            # for trend analysis 
            "content_type": "post",
            "engagement_score": int(row['post_upvotes']) if pd.notna(row['post_upvotes']) else 0,
            "text_length": len(combined_text),
            "was_truncated": len(combined_text) < original_length,
        }
    )
    filtered_points.append(point)
    id += 1

print(f"Filtered points: {len(filtered_points)}")
print(f"Skipped empty: {skipped_empty}")
print(f"Truncated long texts: {truncated_count}")


Filtered points: 5708
Skipped empty: 0
Truncated long texts: 207


In [14]:
# Check how much data we're processing
print(f"Total points to upload: {len(points)}")
print(f"Average text length: {sum(len(p.payload['text']) for p in points) / len(points):.1f} chars")

# Check for very long texts that might cause kernel to die 
long_texts = [p for p in points if len(p.payload['text']) > 5000]
print(f"Posts with >5000 characters: {len(long_texts)}")

Total points to upload: 5708
Average text length: 1148.3 chars
Posts with >5000 characters: 122


In [15]:
# Step 3: Upload with smaller batches and error handling
# Memory usage is reset after each embedding call (Python releases it once the function returns and no references remain).
# take note that you can optimize this as qdrant support batch upsert 
from tqdm import tqdm
import time

batch_size = 25  # Smaller batches
successful_uploads = 0
failed_batches = []

print(f"\nUploading {len(filtered_points)} points in batches of {batch_size}...")

for i in tqdm(range(0, len(filtered_points), batch_size)):
    try:
        batch = filtered_points[i:i + batch_size]
        
        client.upsert(
            collection_name="reddit_post",
            points=batch
        )
        
        successful_uploads += len(batch)
        
        # Small delay to prevent overwhelming
        time.sleep(0.2)
        
    except Exception as e:
        print(f"\n❌ Error uploading batch {i//batch_size + 1}: {e}")
        failed_batches.append(i//batch_size + 1)
        continue

print(f"\n✅ Upload complete!")
print(f"Successful uploads: {successful_uploads}")
print(f"Failed batches: {len(failed_batches)}")

# Verify final count
collection_info = client.get_collection("reddit_post")
print(f"Collection now has {collection_info.points_count} points")


Uploading 5708 points in batches of 25...


100%|███████████████████████████████████████████████████████████████████████████████████████| 229/229 [19:09<00:00,  5.02s/it]


✅ Upload complete!
Successful uploads: 5708
Failed batches: 0
Collection now has 5708 points


In [34]:
# do search 
def search(query, limit=5):


    results = client.query_points(
        collection_name='reddit_post',
        query=models.Document( 
            text=query, # query must be text, qdrant will do the embedding for you 
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    formatted_results = []
    for point in results.points:  # Access points attribute
        formatted_point = {
            'post_title': point.payload['post_title'],
            'post_text': point.payload['post_text'],
            'subreddit': point.payload['subreddit'], 
            'post_url': point.payload['post_url'],
            'post_upvotes': point.payload['post_upvotes']
        }
        formatted_results.append(formatted_point) 

    return formatted_results

In [35]:
test_query = 'What is the most trending topic about Machine Learning?'

search(test_query)

[{'post_title': '[D] What are the bottlenecks holding machine learning back?',
  'post_text': 'I remember this being posted a long, long time ago. What has changed since then? What are the biggest problems holding us back?',
  'subreddit': 'MachineLearning',
  'post_url': 'https://www.reddit.com/r/MachineLearning/comments/1lywxnm/d_what_are_the_bottlenecks_holding_machine/',
  'post_upvotes': 52},
 {'post_title': '[D] What are the bottlenecks holding machine learning back?',
  'post_text': 'I remember this being posted a long, long time ago. What has changed since then? What are the biggest problems holding us back?',
  'subreddit': 'MachineLearning',
  'post_url': 'https://www.reddit.com/r/MachineLearning/comments/1lywxnm/d_what_are_the_bottlenecks_holding_machine/',
  'post_upvotes': 52},
 {'post_title': '[D] What are the bottlenecks holding machine learning back?',
  'post_text': 'I remember this being posted a long, long time ago. What has changed since then? What are the biggest p

In [36]:
# Build Prompt 
def build_prompt(query, search_results):
    prompt_template = """
You're a reddit summariser. Answer user's question based on the CONTEXT given to you.
If you did not spot useful information, then answer based on your own knowledge. 
Otherwise, use only the facts from the CONTEXT when answering the question.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"title: {doc['post_title']}\ncontent: {doc['post_text']}\nurl: {doc['post_url']}\nsubreddit: {doc['subreddit']}\npost_upvotes: {doc['post_upvotes']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [37]:
test_query = 'What is the most trending topic about Machine Learning?'
search_results = search(test_query)
prompt = build_prompt(test_query, formatted_results)
prompt 

"You're a reddit summariser. Answer user's question based on the CONTEXT given to you.\nIf you did not spot useful information, then answer based on your own knowledge. \nOtherwise, use only the facts from the CONTEXT when answering the question.\n\nQUESTION: What is the most trending topic about Machine Learning?\n\nCONTEXT: \ntitle: [D] What are the bottlenecks holding machine learning back?\ncontent: I remember this being posted a long, long time ago. What has changed since then? What are the biggest problems holding us back?\nurl: https://www.reddit.com/r/MachineLearning/comments/1lywxnm/d_what_are_the_bottlenecks_holding_machine/\nsubreddit: MachineLearning\npost_upvotes: 52\n\ntitle: [D] What are the bottlenecks holding machine learning back?\ncontent: I remember this being posted a long, long time ago. What has changed since then? What are the biggest problems holding us back?\nurl: https://www.reddit.com/r/MachineLearning/comments/1lywxnm/d_what_are_the_bottlenecks_holding_mach

In [ ]:
#
#curl 
#    https://api.groq.com/openai/v1/chat/completions -s \ 
#    -H "Content-Type: application/json" \ 
#    -H "Authorization: xxxxxx" \ 
#    -d '{ "model": "meta-llama/llama-4-scout-17b-16e-instruct", 
#        "messages": [{ "role": "user", 
#                        "content": "Explain the importance of fast language models" }] }' 



In [43]:
# LLM 
import os 
import requests 
os.environ["API_KEY"] = "heiheihei"

def lamma3_groq(prompt):
    api_key = os.getenv('API_KEY') 
    url = "https://api.groq.com/openai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    
    data = {
        "model": "llama3-8b-8192",  # or "llama3-70b-8192" for larger model
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 1024
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [45]:
Answer = lamma3_groq(prompt)

'Based on the context, the most trending topic about Machine Learning appears to be the discussion about the bottlenecks holding machine learning back.'

In [49]:
def rag_pipeline(query): 
    search_results = search(query)
    prompt = build_prompt(query, formatted_results)
    answer = lamma3_groq(prompt)
    return answer 

In [50]:
test_query = 'What are people discussing about GROP now?'
rag_pipeline(test_query) 

'Based on the context provided, it appears that there is no discussion about "GROP" in this thread. The title and content of the post are about the bottlenecks holding machine learning back, and there is no mention of "GROP" in the text.'

In [51]:
test_query = 'What are some common machine learning models reddit users discuss?'
rag_pipeline(test_query) 

"Based on the context provided, it appears that the post is about discussing the bottlenecks holding back machine learning. Since it's a discussion post, there are no specific models mentioned. However, I can provide some common machine learning models that are often discussed by Reddit users in the MachineLearning subreddit:\n\n1. Neural Networks: A type of machine learning model inspired by the structure and function of the human brain.\n2. Decision Trees: A simple, tree-based model used for classification and regression tasks.\n3. Random Forests: An ensemble learning method that combines multiple decision trees to improve accuracy.\n4. Support Vector Machines (SVMs): A type of supervised learning algorithm that separates classes by creating a hyperplane.\n5. Gradient Boosting: A popular ensemble learning method that combines multiple weak models to create a strong one.\n6. K-Means: An unsupervised learning algorithm for clustering data points into groups.\n7. Linear Regression: A ty

In [52]:
test_query = 'What are people discussing about GROP now?'
search_results = search(test_query)
search_results

[{'post_title': 'Reminder: Long COVID study group today 6PM EST',
  'post_text': "Hello friends! A reminder that we are still doing our long COVID study group and if you want to check us out, we are having a session today at 6PM EST. Two papers will be presented by two different participants. If you want to join, let me know and I'll add you to the group chat where you will be able to access the Zoom link just before the meeting. \n\nRules:\n\n* No COVID deniers or conspiracy theorists\n* No conversation about the origins of COVID\n* No anti-science, anti-medicine rhetoric\n* No anti-vaxxers\n* No racism, sexism, ableism, homophobia, transphobia, or bigotry of any kind\n* No harassing or bullying any other members of the group. Don’t be a jerk.\n* Be respectful\n* This is a study group and not a support group. This time will be spent discussing scientific research. If you are looking for emotional support, or a place to vent, I recommend trying to find long covid support groups.\n* Pre